 # Aktualizacja bazy danych
 W tym miejscu odpowiednio skonfiguruj połączenie do bazy danych.

 Tutaj zaimportuj potrzebne biblioteki

In [22]:
import psycopg2
from psycopg2 import connect, extensions, sql
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import matplotlib.pyplot as plt
import os


 ## Konfiguracja połączenia
 Tutaj uzupełnij konfigurację połączenia

In [23]:
username = 'postgres'
password = 'password'

host = 'localhost'
database = 'airlines'
port = 5432

 Tutaj zdefiniuj zmienną `con` oraz `cursor`

In [24]:
conn = psycopg2.connect(
    host=host,
    database=database,
    user=username,
    password=password,
    port=port)
cursor = conn.cursor()

 ## Wczytanie pliku `reporting.sql`
 Z katalogu `sql` wczytaj plik `reporting.sql`

In [25]:
with open(r'C:\Users\admin\Desktop\Koncowego_sprawdzony\sql\reporting.sql', mode='r') as sql_file:
    reporting_sql = sql_file.read()
    
print(reporting_sql)


DROP SCHEMA IF EXISTS reporting CASCADE;
CREATE SCHEMA IF NOT EXISTS reporting;


CREATE OR REPLACE VIEW reporting.flight AS
SELECT *,
    CASE
           WHEN flight.dep_delay_new > 0 THEN 1
           ELSE 0
       END AS is_delayed
	
FROM
    flight
WHERE
    flight.cancelled = 0;

CREATE OR REPLACE VIEW reporting.top_reliability_roads
 AS
 SELECT f.origin_airport_id,
    al1.name AS origin_airport_name,
    f.dest_airport_id,
    al2.name AS dest_airport_name,
    f.year,
    count(*) AS cnt,
    round(avg(
        CASE
            WHEN f.dep_delay_new > 0::double precision THEN 1
            ELSE 0
        END), 2) AS reliability,
    rank() OVER (ORDER BY (avg(
        CASE
            WHEN f.dep_delay_new > 0::double precision THEN 1
            ELSE 0
        END)) DESC) AS nb
   FROM flight f
     JOIN airport_list al1 ON f.origin_airport_id = al1.origin_airport_id
     JOIN airport_list al2 ON f.dest_airport_id = al2.origin_airport_id
  GROUP BY f.origin_airport_id, al1.name

 W tym miejscu odpowiednio rozdziel zawartość pliku `reporting.sql` na mniejsze kwerendy używając `;`

In [26]:
queries = reporting_sql.split(';')
print(queries)

['\nDROP SCHEMA IF EXISTS reporting CASCADE', '\nCREATE SCHEMA IF NOT EXISTS reporting', '\n\n\nCREATE OR REPLACE VIEW reporting.flight AS\nSELECT *,\n    CASE\n           WHEN flight.dep_delay_new > 0 THEN 1\n           ELSE 0\n       END AS is_delayed\n\t\nFROM\n    flight\nWHERE\n    flight.cancelled = 0', '\n\nCREATE OR REPLACE VIEW reporting.top_reliability_roads\n AS\n SELECT f.origin_airport_id,\n    al1.name AS origin_airport_name,\n    f.dest_airport_id,\n    al2.name AS dest_airport_name,\n    f.year,\n    count(*) AS cnt,\n    round(avg(\n        CASE\n            WHEN f.dep_delay_new > 0::double precision THEN 1\n            ELSE 0\n        END), 2) AS reliability,\n    rank() OVER (ORDER BY (avg(\n        CASE\n            WHEN f.dep_delay_new > 0::double precision THEN 1\n            ELSE 0\n        END)) DESC) AS nb\n   FROM flight f\n     JOIN airport_list al1 ON f.origin_airport_id = al1.origin_airport_id\n     JOIN airport_list al2 ON f.dest_airport_id = al2.origin_ai

 W tym miejscu wykonaj każdą z kwerend, aby zainicjować strukturę bazy danych

In [27]:
with conn.cursor() as cur:
    for query in queries:
        if query.strip() != '':
            cur.execute(query)
    conn.commit()

In [28]:
conn.close()